In [1]:
import sys
import os
import pandas

In [2]:
# Se verfica entorno de ejecución
in_colab = "google.colab" in sys.modules

if in_colab:
    from google.colab import drive

    drive.mount("/content/drive")
    BASE_DIR = "/content/drive/My Drive/Diplo2020 Mentoria/"
    
    # Instala pdfminer 
    !pip install pdfminer
else:
    BASE_DIR = "../"

In [3]:
train_data_dir = BASE_DIR + "Datos/Electoral/"
train_data = BASE_DIR + "Corpus3.cor"

# Función para obtener las lista de documentos de jurisprudencia para procesar

In [4]:
def getListOfFilesJurisprudencia(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = dirName + '/' + entry
        tipo = 'SENTENCIA'
        if entry[0] == 'A':
          tipo = 'AUTO'
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFilesJurisprudencia(fullPath)
        else:
            allFiles.append([fullPath, tipo])
                
    return allFiles

# Función para leer PDF

In [5]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [6]:
def pdf_to_text(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    layout = LAParams() 
    device = TextConverter(rsrcmgr, retstr, laparams=layout)    
    filepath = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    
    for page in PDFPage.get_pages(filepath, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    filepath.close()
    device.close()
    retstr.close()
    return text

# Función de normalización

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jperezv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jperezv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
def normalize_text(text):
  # Tokenización
  tokenizador = ToktokTokenizer()
  tokens = tokenizador.tokenize(text)

  # Paso a minúscula
  tokens = [word.lower() for word in tokens]

  # Eliminamos símbolos de puntuación
  tokens = list(filter(lambda token: nltk.tokenize.punkt.PunktToken(token).is_non_punct, tokens)) 

  # Eliminamos stopwords
  tokens = list(filter(lambda token: token not in stopwords.words('spanish'), tokens)) 

  # Sustituir palabras con demasiada variabilidad (p.ej. números)
  dic_Igualdades = {'nro.':'nro', 'n°':'nro', 'n.°':'nro', 'nº':'nro', 'n.º':'nro', 'n.':'nro', 'art.':'artículo', 'articulo':'artículo'}
  for i in range(len(tokens)):    
      for key in dic_Igualdades:
          if tokens[i] == key:
              tokens[i] = dic_Igualdades[key]

  # Lemmatización
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  return ' '.join(tokens)

# Genero el corpus

In [11]:
listOfFileJurisprudencia = getListOfFilesJurisprudencia(train_data_dir)
print('Cantidad de archivos a importar:', len(listOfFileJurisprudencia))    

Cantidad de archivos a importar: 151


In [12]:
doc = listOfFileJurisprudencia[1:2]

In [13]:
texto = pdf_to_text(doc[0][0])
tipo = doc[0][1]
tipo

'AUTO'

In [14]:
texto_norm = normalize_text(texto)
print(texto_norm)

sala electoral comp.originaria tribunal superior protocolo auto nro resolución 2 año 2018 tomo 1 folio 6-8 expediente 3412648 denuncia formulada roberto roque ramirez cuestion competencia denuncia formulada auto numero 2. cordoba vistos auto caratulados denuncia formulada roberto roque ramirez cuestion competencia expte. n 3412648 traídos despacho fine resolver presunto conflicto competencia surgido juzgado control niñez juventud faltas secretaría control faltas río tercero juzgado electoral provincia 1 ciudadano comuna general fotheringham dn. roberto roque ramírez formuló denuncia presidente comunal señor daniel aurelio stobbia alegando último viviría pueblo llevaría ejerciendo cuarto mandato tal conducta criterio denunciante encuadraría artículo 248 código penal cfr. fs. vta. 2 señora fiscal instrucción primer turno segunda nominación ciudad río tercero mediante proveído fecha 7 octubre 2016 fundándose características supuesto hecho delictivo entiende corresponde régimen electoral a

In [15]:
corpus3 = []
for file in listOfFileJurisprudencia:
  if file[0].endswith(".pdf"):
    texto = normalize_text(pdf_to_text(file[0]))
    corpus3.append([texto, file[0], file[1]])


In [16]:
df_corpus3 = pandas.DataFrame(corpus3,columns=['TEXTO','DOCUMENTO','TIPO'])

In [17]:
df_corpus3.head()

,TEXTO,DOCUMENTO,TIPO
0,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 001-2018 COOPI C MUN CAR...,AUTO
1,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 002-2018 Denuncia R R RA...,AUTO
2,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 003-2018 VEDIA FLORES Cr...,AUTO
3,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 004-2018 FERNANDEZ Favio...,AUTO
4,sala electoral comp.originaria tribunal superi...,../Datos/Electoral//A 005-2018 ATANOR ADI ref.pdf,AUTO


In [18]:
df_corpus3.to_csv(BASE_DIR + "corpus3.csv")